In [183]:
import numpy as np 
import pandas as pd

import matplotlib.pyplot as plt
from numpy.random import dirichlet
from scipy.special import logsumexp
from sklearn.decomposition import PCA
from sklearn.metrics import adjusted_rand_score

# read data

In [184]:
X_df = pd.read_csv("./data/Q3/mixture1.geno", header=None).values
Xnew_df = pd.read_csv("./data/Q3/mixture2.geno", header=None).values
F_df = pd.read_csv("./data/Q3/mixture1.freq", header=None).values
Z_df = pd.read_csv("./data/Q3/mixture1.ganc", header=None).values

# implementation M

In [185]:
#X: N by M data matrix
#gamma: P(Z_i=k | X_i=x_i, theta) N by K
def M_step(X, gamma): 
    N, M = X.shape
    K = gamma.shape[1]
    
    ######### TODO 3a: modify the following to have meaning updates #########
    pis = np.empty(K)
    F = np.empty(shape=(M, K)) # frequency matrix: M by K

    # Update pi -- pi[k] = proportion of all individuals coming from population k
    for k in range(K):
        pis[k] = np.mean(gamma[:, k])


    # Iterate through each population
    for k in range(K):
        partial_ass_sum = np.sum(gamma[:, k]) # sum up partial assignments across all individuals in population (# individuals in population)
        # Iterate through each SNP
        for j in range(M):
            if partial_ass_sum == 0:
                F[j][k] = 0
            else:
                # Iterate through each individual
                partial_ass_sum_SNP_j = 0 # num of 1 genos at SNP j across all individuals belomnging to population 
                for i in range(N):
                    partial_ass_sum_SNP_j += gamma[i][k] * X[i][j] # Multiply soft assignment to population k for individual i by genotype
                F[j][k] = partial_ass_sum_SNP_j / partial_ass_sum
    
    
    ######### End of modification #########     
    return ({"pis": pis, "F": F})

gamma = Z_df
result = M_step(X_df, gamma)
# print (result)
print (result["pis"])
print (result["F"])


[0.394 0.606]
[[0.79187817 0.00660066]
 [0.63959391 0.00825083]
 [0.24619289 0.5379538 ]
 [0.27664975 0.34488449]
 [0.08375635 0.36963696]
 [0.22335025 0.03630363]
 [0.33756345 0.49339934]
 [0.49746193 0.48514851]
 [0.37817259 0.62706271]
 [0.59390863 0.34983498]
 [0.35532995 0.05610561]
 [0.1928934  0.21122112]
 [0.04822335 0.2359736 ]
 [0.17258883 0.97689769]
 [0.43147208 0.53630363]
 [0.10152284 0.02145215]
 [0.35532995 0.74587459]
 [0.46700508 0.3960396 ]
 [0.32994924 0.11386139]
 [0.24873096 0.52475248]
 [0.50507614 0.4950495 ]
 [0.35279188 0.15841584]
 [0.28680203 0.72607261]
 [0.14467005 0.47524752]
 [0.65736041 0.23927393]
 [0.29695431 0.48514851]
 [0.36040609 0.36963696]
 [0.58883249 0.47854785]
 [0.10152284 0.11551155]
 [0.35532995 0.00660066]
 [0.32741117 0.45214521]
 [0.06598985 0.1039604 ]
 [0.02538071 0.15676568]
 [0.45685279 0.56270627]
 [0.04060914 0.20132013]
 [0.13451777 0.71287129]
 [0.4035533  0.57590759]
 [0.40862944 0.51485149]
 [0.5964467  0.35478548]
 [0.1192893

# implementation E

In [186]:
#X: N by M data matrix
#params: a dictionary with two parameters returned from M_step
def E_step(X, params, thr=10**(-8)):
    F = params["F"] # frequency matrix F: M by K
    pis = params["pis"] # proportion vector pi: length K

    N, M = X.shape
    K = F.shape[1] 

    ######### TODO 3b: modify the following to have meaning updates #########
    #calculate weighted_log_prob: log(P(X_i=x_i | Z_i=k, theta) * P(Z_i=k | theta))
    #calcualte log_prob_sample: log P(Xi=x_i | theta) length N vector. Hint: use logsumexp function
    #calcualte log_prob_data: log P(X_1:n=x_1:n | theta) scalar
    #calculate log_gammas: log P(Z_i=k | X_i=x_i, theta) N by K
    weighted_log_prob = np.zeros((N, K))
    for i in range(N):
        for k in range(K):
            prod = 1
            for j in range(M):
                allele_freq = thr
                if F[j][k] != 0:
                    allele_freq = F[j][k]
                one_minus = thr
                if 1 - allele_freq != 0:
                    one_minus = 1 - allele_freq
                prod *= (allele_freq ** X[i][j]) * (one_minus ** (1 - X[i][j]))
            weighted_log_prob[i][k] = np.log(prod) + np.log(pis[k])

    log_prob_sample = np.zeros(N)
    for i in range(N):
        log_prob_sample[i] = logsumexp(weighted_log_prob[i])

    log_prob_data = np.sum(log_prob_sample)

    log_gammas = np.zeros((N, K))
    for k in range(K):
        for i in range(N):
            log_gammas[i][k] = weighted_log_prob[i][k] - log_prob_sample[i]

    ######### end of modification #########
    return log_gammas, log_prob_data

params = {"F": F_df, "pis": result["pis"]}
E_results = E_step(X_df, params)
log_gammas = E_results[0]

N = X_df.shape[0]

predicted_ancestry = np.empty(N)
for i in range(N):
    predicted_ancestry[i] = np.argmax(log_gammas[i])

actual_ancestry = np.empty(N)
for i in range(N):
    actual_ancestry[i] = np.argmax(Z_df[i])

print (adjusted_rand_score(actual_ancestry, predicted_ancestry))



# implementation EM 

In [187]:
def EM(X, K = 2, max_iter = 100, tol = 10**(-4), n_init = 3, debug = False):
    
    N, M = X.shape 
    res = {}
    best_log_prob_data = -np.inf
    converged  = False 

    predicted_ancestries = []
    best_pis = None

    #loop through different random starting points
    for init in range(1, 1+n_init, 1):
        np.random.seed(init)
        if debug:
            print(f"starting EM on random initialization: {init} out of {n_init}")
        
        ######### TODO 3c: modify the following to have the full EM updates #########
        
        #initialize soft assignment 
        gammas = np.empty(shape=(N, K))
        start_gammas = gammas
        
        for i in range(N):
            gammas[i] = dirichlet([1] * K)

        
        
        log_prob_data = -np.inf
        all_log_likelihoods = []
        for n_iter in range(1, 1+max_iter, 1):
            prev_log_prob_data = log_prob_data

            M_results = M_step(X, gammas) # pis and allele frequency matrices
            
            log_gammas, log_prob_data = E_step(X, M_results)

            all_log_likelihoods.append(log_prob_data)

            gammas = np.exp(log_gammas)
        
            ######### convergence check #########
            change = (log_prob_data - prev_log_prob_data) / N
            if abs(change) < tol:
                if debug:
                    print(f"random initialization {init} converged at iteration {n_iter}")
                    print("")
                converged = True
                break
        
        plt.scatter([i for i in range(len(all_log_likelihoods))], all_log_likelihoods)
        plt.title(f"Log Likelihood vs Iteration, for Iteration {init}")
        plt.xlabel("Iteration")
        plt.ylabel("Log Likelihood")
        plt.show()

        ######### update on the best initialization #########
        if log_prob_data > best_log_prob_data:
            best_log_prob_data = log_prob_data
            best_init = start_gammas
            best_pis = M_results["pis"]

        print (f"Iteration {init}")
        print (f"Best init gamma: {best_init}")
        print (f"Best log likelihood: {best_log_prob_data}")

        # best_init = NULL

        predicted_ancestry = np.empty(N)
        for i in range(N):
            predicted_ancestry[i] = np.argmax(gammas[i])
        predicted_ancestries.append(predicted_ancestry)
         
        
    ######### end of modification #########
    res["converged"] = converged     
    res["best_init"] = best_init
    res["predicted_ancestries"] = predicted_ancestries
    res["best_log_likelihood"] = best_log_prob_data  
    res["best_pis"] = best_pis
    return res

In [188]:
X = X_df[:, [i for i in range(10)]] # use first 10 SNPs
result = EM(X, K=3, debug=True)
predicted_ancestries = result["predicted_ancestries"]

In [189]:
pca = PCA(n_components=2)
pca.fit(X_df)
reduced = pca.transform(X_df)

pca1, pca2 = [l[0] for l in reduced], [l[1] for l in reduced]

In [190]:
import random

for i, pred_ances in enumerate(predicted_ancestries):
    print (pred_ances)
    # countries = [l[1] for l in y]
    unique_ancestries = list(set(pred_ances))
    n = len(unique_ancestries)

    data = {"pca2": pca2, "pca1": pca1, "ancestry": pred_ances}

    df = pd.DataFrame(data)

    color_list = ["#%06x" % random.randint(0, 0xFFFFFF) for _ in range(n)]
    color_map = {uc: c for uc, c in zip(unique_ancestries, color_list)}

    for ancestry, group in df.groupby("ancestry"):
        scatter = plt.scatter(group["pca2"], group["pca1"], label=ancestry, color=color_map[ancestry])

    plt.xlabel("pc2")
    plt.ylabel("pc1")
    plt.title(f"PC1 vs PC2, iteration {i}")
    plt.legend()

    plt.show()

In [191]:
actual_ancestry = np.empty(N)
for i in range(N):
    actual_ancestry[i] = np.argmax(Z_df[i])

snp_cts = [10, 25, 50, 100, 150, 200, 250]
all_scores = []
for snp_ct in snp_cts:
    X = X_df[:, [i for i in range(snp_ct)]] # use first 10 SNPs
    result = EM(X, K=2)

    predicted_ancestries = result["predicted_ancestries"]
    best_predicted_ancestry = predicted_ancestries[-1]
    score = adjusted_rand_score(actual_ancestry, best_predicted_ancestry)
    all_scores.append(score)

plt.scatter(snp_cts, all_scores)
plt.title("Adjusted Rand Score vs SNP Count, for K = 2")
plt.xlabel("SNP Count")
plt.ylabel("Adjusted Rand Score")
plt.show()

In [192]:
components = [1, 2, 3, 4, 5]
best_log_likelihoods = []
for k in components:
    X = Xnew_df[:, [i for i in range(100)]] # use first 100 SNPs
    result = EM(X, K=k)
    best_log_likelihoods.append(result["best_log_likelihood"])

plt.scatter(components, best_log_likelihoods)
plt.xlabel("Component Number")
plt.ylabel("Log Likelihood")
plt.title("Log Likelihood vs. Component Number")
plt.show()

In [193]:
result = EM(Xnew_df, K=2)
print (result["best_pis"])

Iteration 1
Best init gamma: [[2.97511489e-01 7.02488511e-01]
 [3.17613829e-04 9.99682386e-01]
 [6.20945430e-01 3.79054570e-01]
 ...
 [7.28502864e-01 2.71497136e-01]
 [1.03401610e-01 8.96598390e-01]
 [1.06242518e-02 9.89375748e-01]]
Best log likelihood: -129163.5994772429
Iteration 2
Best init gamma: [[2.97511489e-01 7.02488511e-01]
 [3.17613829e-04 9.99682386e-01]
 [6.20945430e-01 3.79054570e-01]
 ...
 [7.28502864e-01 2.71497136e-01]
 [1.03401610e-01 8.96598390e-01]
 [1.06242518e-02 9.89375748e-01]]
Best log likelihood: -129163.5994772429
Iteration 3
Best init gamma: [[2.97511489e-01 7.02488511e-01]
 [3.17613829e-04 9.99682386e-01]
 [6.20945430e-01 3.79054570e-01]
 ...
 [7.28502864e-01 2.71497136e-01]
 [1.03401610e-01 8.96598390e-01]
 [1.06242518e-02 9.89375748e-01]]
Best log likelihood: -129163.5994772429


NameError: name 'results' is not defined